In [63]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import plotly.graph_objects as go

In [64]:
df = pd.read_csv('df_alquileres_ordenado.csv')

In [74]:
df = df.dropna(subset=['precio'])

In [75]:
df.shape

(11576, 57)

In [65]:
# Seleccionamos las columnas que incluiremos en el radar (numericas), ademas añadimos el "identificador" para buscar y "ubicacion" como etiqueta
columnas_importantes = ['identificador', 'superficie', 'superficie_construida', 'superficie_util', 'planta', 'baños', 'precio_m2', 'precio', 'habitaciones', 'ubicacion']
df_importante = df[columnas_importantes].copy()

In [66]:
# Escalamos el precio por separado porque es el valor notablemente mas extremo que desfigura el grafico
scaler_price = RobustScaler()
df_importante['precio_escalado'] = scaler_price.fit_transform(df_importante[['precio']])

# Normalizamos el resto
scaler_others = MinMaxScaler()
df_others_scaled = scaler_others.fit_transform(df_importante.drop(columns=['identificador', 'precio', 'precio_escalado', 'ubicacion']))

# Los combinamos en un unico df
df_scaled = pd.DataFrame(df_others_scaled, columns=columnas_importantes[1:-2])
df_scaled['precio'] = df_importante['precio_escalado']
df_scaled['ubicacion'] = df_importante['ubicacion'].values
df_scaled['identificador'] = df_importante['identificador'].values

In [67]:
df_scaled['identificador'].unique()

array(['40920273409_100500', '15062564089_100500', '658149002_501470',
       ..., '43392414912_109800', '945856909436630_109800',
       '44201590595_109800'], dtype=object)

In [68]:
# Introducimos manualmente los identificadores a comparar
id1 = '51683300341_529013'
id2 = '51722188486_518438'

In [69]:
# Filtramos
piso1 = df_scaled[df_scaled['identificador'] == id1].drop(columns=['identificador'])
piso2 = df_scaled[df_scaled['identificador'] == id2].drop(columns=['identificador'])

In [70]:
# Creamos el gráfico de radar
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=piso1.iloc[0],
    theta=piso1.columns,
    fill='toself',
    name=f'{id1} - {piso1["ubicacion"].values[0]}'
))

fig.add_trace(go.Scatterpolar(
    r=piso2.iloc[0],
    theta=piso2.columns,
    fill='toself',
    name=f'{id2} - {piso2["ubicacion"].values[0]}'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-2, 2]
        )
    ),
    showlegend=True
)

fig.show()

Aquí tenemos el grafico comparativo en forma de radar entre las principales caracteristicas de los inmuebles seleccionados.
Ahora añadiremos un df exclusivo de los dos comparados para ver los valores reales de cada categoria esclada.

In [71]:
id1 = '51683300341_529013'
id2 = '51722188486_518438'
piso1_original = df[df['identificador'] == id1][columnas_importantes]
piso2_original = df[df['identificador'] == id2][columnas_importantes]

df_comparativo = pd.concat([piso1_original, piso2_original], ignore_index=True)

In [72]:
df_comparativo

,identificador,superficie,superficie_construida,superficie_util,planta,baños,precio_m2,precio,habitaciones,ubicacion
0,51683300341_529013,79.0,79.0,64.0,2.0,1.0,8.0,650.0,2.0,roquetas_de_mar
1,51722188486_518438,15.0,15.0,14.0,0.0,1.0,30.0,450.0,1.0,cartagena
